# Multi-layer perceptron (classification)

In [1]:
using MLDatasets
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using Statistics: mean

## Load data

In [2]:
train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata()

(FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

FixedPointNumbers.Normed{UInt8,8}[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

## Preprocessing

In [3]:
train_x = train_x / 255
train_x = hcat([vec(train_x[:, :, k]) for k in 1:size(train_x, 3)]...)

784×60000 Array{Float32,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [4]:
train_y = onehotbatch(train_y, 0:9)

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

In [5]:
dataset = repeated((train_x, train_y), 200)

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{Array{Float32,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}}(Base.Iterators.Repeated{Tuple{Array{Float32,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[false true … false false; false false … false false; … ; false false … false true; false false … false false])), 200)

## Model

In [6]:
m = Chain(
    Dense(28^2, 32, relu),
    Dense(32, 10),
    softmax)

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

In [7]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

In [8]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

accuracy (generic function with 1 method)

## Optimizer

In [9]:
evalcb() = @show(loss(train_x, train_y))
opt = ADAM(0.005)

ADAM(0.005, (0.9, 0.999), IdDict{Any,Any}())

In [10]:
@time Flux.train!(loss, params(m), dataset, opt, cb=throttle(evalcb, 10))

loss(train_x, train_y) = 2.3013847f0 (tracked)
loss(train_x, train_y) = 2.286893f0 (tracked)
loss(train_x, train_y) = 2.258938f0 (tracked)
loss(train_x, train_y) = 2.2147746f0 (tracked)
loss(train_x, train_y) = 2.1515112f0 (tracked)
loss(train_x, train_y) = 2.0696058f0 (tracked)
loss(train_x, train_y) = 1.9959438f0 (tracked)
loss(train_x, train_y) = 1.9237522f0 (tracked)
loss(train_x, train_y) = 1.839273f0 (tracked)
loss(train_x, train_y) = 1.743771f0 (tracked)
loss(train_x, train_y) = 1.6481404f0 (tracked)
loss(train_x, train_y) = 1.5463043f0 (tracked)
loss(train_x, train_y) = 1.4492829f0 (tracked)
loss(train_x, train_y) = 1.3732666f0 (tracked)
loss(train_x, train_y) = 1.2953727f0 (tracked)
loss(train_x, train_y) = 1.2236638f0 (tracked)
loss(train_x, train_y) = 1.163712f0 (tracked)
loss(train_x, train_y) = 1.1137862f0 (tracked)
loss(train_x, train_y) = 1.0723995f0 (tracked)
258.117222 seconds (22.73 M allocations: 49.743 GiB, 21.62% gc time)


In [11]:
accuracy(train_x, train_y)

0.7762666666666667

In [12]:
# Test set accuracy
test_x = test_x / 255
test_x = hcat([vec(test_x[:, :, k]) for k in 1:size(test_x, 3)]...)
test_y = onehotbatch(test_y, 0:9)

10×10000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false  false  false  false
 false  false   true  false  false     false  false  false  false  false
 false   true  false  false  false      true  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false  false  false   true     false  false   true  false  false
 false  false  false  false  false  …  false  false  false   true  false
 false  false  false  false  false     false  false  false  false   true
  true  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false

In [13]:
accuracy(test_x, test_y)

0.7841